## Import librarys

In [280]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)
sns.set(style='white', context='notebook', palette='deep')

## Load data

In [352]:
df = pd.read_csv("./input/2018.csv")
df_extra = pd.read_csv("./input/df_with_neis.csv")

In [353]:
df_extra.drop(['vehicle_id','shiryun_cat','avg_dist','billing_minutes','orig_addr','dest_addr',
               'start_dt','end_dt','orig_addr_c','dest_addr_c','orig_x_merc',
               'orig_y_merc','dest_x_merc','dest_y_merc','start_hour','start_dayofweek','start_month'],axis=1,inplace=True)

In [354]:
df = df.rename({'מספר רכב': 'Car id', 'מספר הזמנה': 'Order number','קטגוריית שריון':'Category',
                 'ממוצע מרחק לנסיעה':'Avg distance','דקות לחיוב':'Billing minutes','כתובת התחלה':'Start Address',
                 'כתובת סיום':'Dest Address','תאריך ושעה התחלת שריון':'Start time','תאריך ושעה סיום הזמנה':'End time'},
                axis=1) 

In [355]:
df_extra = df_extra.rename({'order_id': 'Order number'},axis=1) 

In [356]:
df = pd.merge(df, df_extra, on=['Order number'], how='left')

## Data Preperation

### Create new rows from End Address and End time

In [358]:
df_temp = df.copy()

In [359]:
df.drop(["Dest Address","End time","dest_coord","dest_x","dest_y","dest_nei"],axis=1,inplace=True)
df_temp.drop(["Start Address","Start time","orig_coord","orig_x","orig_y","orig_nei"],axis=1,inplace=True)

In [360]:
df = df.rename({"Start Address":"Address","Start time": "Time","orig_coord":"Coords","orig_x":"Lat","orig_y":"Lon","orig_nei":"neighborhood",},axis=1)
df_temp = df_temp.rename({"Dest Address":"Address","End time": "Time","dest_coord":"Coords","dest_x":"Lat","dest_y":"Lon","dest_nei":"neighborhood"},axis=1)

In [361]:
df = pd.concat([df,df_temp])

In [362]:
df.sort_values(["Order number","Time"],axis=0,inplace=True)

In [363]:
df.head()

,Car id,Order number,Category,Avg distance,Billing minutes,Address,Time,kmh,Coords,Lat,Lon,neighborhood
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל",23/10/2017 0:08,37.151703,"(32.0807428, 34.7792228)",32.080743,34.779223,הצפון הישן-החלק הדרומי
0,46-525-81,60386,שריון חינמי,8,13,"הרב רובינשטיין 61, תל אביב יפו, ישראל",23/10/2017 0:31,37.151703,"(32.0357011, 34.7581719)",32.035701,34.758172,(יפו ד' (גבעת התמרים
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל",23/10/2017 0:09,21.556886,"(32.1050698, 34.7892274)",32.105070,34.789227,'תכנית ל
1,48-437-81,60387,שריון חינמי,3,8,"ז'בוטינסקי 10, תל אביב יפו, ישראל",23/10/2017 0:21,21.556886,"(32.0895831, 34.7734306)",32.089583,34.773431,הצפון הישן - החלק הצפוני
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל",23/10/2017 0:10,0.000000,"(32.0836176, 34.7710507)",32.083618,34.771051,הצפון הישן-החלק הדרומי


### check for null and missing values

In [364]:
# Fill empty and NaNs values with NaN
df = df.fillna(np.nan)

In [365]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300004 entries, 0 to 150001
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Car id           300004 non-null  object 
 1   Order number     300004 non-null  int64  
 2   Category         300004 non-null  object 
 3   Avg distance     300004 non-null  int64  
 4   Billing minutes  275694 non-null  object 
 5   Address          286966 non-null  object 
 6   Time             286358 non-null  object 
 7   kmh              268428 non-null  float64
 8   Coords           272662 non-null  object 
 9   Lat              272662 non-null  float64
 10  Lon              272662 non-null  float64
 11  neighborhood     247720 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 21.7+ MB


Car id                 0
Order number           0
Category               0
Avg distance           0
Billing minutes    24310
Address            13038
Time               13646
kmh                31576
Coords             27342
Lat                27342
Lon                27342
neighborhood       52284
dtype: int64

### Splitting address to country, city, street

In [366]:
split_col = df["Address"].str.split(",", expand=True)
split_col[:3]

,0,1,2,3
0,גורדון 86-88,תל אביב יפו,ישראל,None
0,הרב רובינשטיין 61,תל אביב יפו,ישראל,None
1,יעקב אפטר 3,תל אביב יפו,ישראל,None


In [367]:
df["Street"] = split_col[0]
df["City"] = split_col[1]
df["Country"] = split_col[2]

#### extracting street without numbers

In [368]:
df["Street_c"] = df['Street'].str.replace(r'(\d+)(-(\d+))*', r'')

#### extract clean address for google api use

In [369]:
df['Address_c'] = df['Address'].str.replace(r'(\d+)-(\d+)', r'\1')

### Extract date and time

In [370]:
df["Time"] = pd.to_datetime(df["Time"])

In [371]:
# df["Date"] = df["Time"].dt.date
# df["time"] = df["Time"].dt.time
df["Year"] = df["Time"].dt.year
df["Month"] = df["Time"].dt.month
df["Day"] = df["Time"].dt.day
df["Hour"] = df["Time"].dt.hour
df["Minute"] = df["Time"].dt.minute

## Visualization

In [372]:
counts = df['neighborhood'].value_counts()
df[df['neighborhood'].isin(counts[counts > 1000].index)]

,Car id,Order number,Category,Avg distance,Billing minutes,Address,Time,kmh,Coords,Lat,Lon,neighborhood,Street,City,Country,Street_c,Address_c,Year,Month,Day,Hour,Minute
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל",2017-10-23 00:08:00,37.151703,"(32.0807428, 34.7792228)",32.080743,34.779223,הצפון הישן-החלק הדרומי,גורדון 86-88,תל אביב יפו,ישראל,גורדון,"גורדון 86, תל אביב יפו, ישראל",2017.0,10.0,23.0,0.0,8.0
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל",2017-10-23 00:09:00,21.556886,"(32.1050698, 34.7892274)",32.105070,34.789227,'תכנית ל,יעקב אפטר 3,תל אביב יפו,ישראל,יעקב אפטר,"יעקב אפטר 3, תל אביב יפו, ישראל",2017.0,10.0,23.0,0.0,9.0
1,48-437-81,60387,שריון חינמי,3,8,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017-10-23 00:21:00,21.556886,"(32.0895831, 34.7734306)",32.089583,34.773431,הצפון הישן - החלק הצפוני,ז'בוטינסקי 10,תל אביב יפו,ישראל,ז'בוטינסקי,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017.0,10.0,23.0,0.0,21.0
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל",2017-10-23 00:10:00,0.000000,"(32.0836176, 34.7710507)",32.083618,34.771051,הצפון הישן-החלק הדרומי,רופין 32-34,תל אביב יפו,ישראל,רופין,"רופין 32, תל אביב יפו, ישראל",2017.0,10.0,23.0,0.0,10.0
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל",2017-10-23 00:15:00,0.000000,"(32.0836176, 34.7710507)",32.083618,34.771051,הצפון הישן-החלק הדרומי,רופין 32-34,תל אביב יפו,ישראל,רופין,"רופין 32, תל אביב יפו, ישראל",2017.0,10.0,23.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136328,63-889-81,224600,שריון חינמי,4,20,"שלמה המלך 41, תל אביב יפו, ישראל",2018-04-16 23:31:00,11.952191,"(32.0794621, 34.7769013)",32.079462,34.776901,פלורנטין,שלמה המלך 41,תל אביב יפו,ישראל,שלמה המלך,"שלמה המלך 41, תל אביב יפו, ישראל",2018.0,4.0,16.0,23.0,31.0
136329,63-875-81,224601,שריון חינמי,0,1,"בורוכוב 4, תל אביב יפו, ישראל",2018-04-16 23:05:00,0.000000,"(32.0721286, 34.7745174)",32.072129,34.774517,הצפון הישן - החלק הצפוני,בורוכוב 4,תל אביב יפו,ישראל,בורוכוב,"בורוכוב 4, תל אביב יפו, ישראל",2018.0,4.0,16.0,23.0,5.0
136329,63-875-81,224601,שריון חינמי,0,1,"בורוכוב 4, תל אביב יפו, ישראל",2018-04-16 23:07:00,0.000000,"(32.0721286, 34.7745174)",32.072129,34.774517,נוה אביבים וסביבתה,בורוכוב 4,תל אביב יפו,ישראל,בורוכוב,"בורוכוב 4, תל אביב יפו, ישראל",2018.0,4.0,16.0,23.0,7.0
136330,46-539-81,224602,שריון חינמי,3,9,"אליהו בית צורי 28-30, תל אביב יפו, ישראל",2018-04-16 23:05:00,21.101993,"(32.1241609, 34.8007742)",32.124161,34.800774,הצפון הישן-החלק הדרומי,אליהו בית צורי 28-30,תל אביב יפו,ישראל,אליהו בית צורי,"אליהו בית צורי 28, תל אביב יפו, ישראל",2018.0,4.0,16.0,23.0,5.0
